<a href="https://colab.research.google.com/github/tushant-akar/CS367-Artifical-Intelligence-Lab/blob/main/Marble_Solatire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import heapq
import time
from collections import deque

In [4]:
class Node:
    def __init__(self, board, g=0, h=0, path=None):
        self.board = board
        self.g = g
        self.h = h
        self.f = g + h
        self.path = path if path is not None else []

    def __lt__(self, other):
        return self.f < other.f

directions = [(0, -2), (0, 2), (2, 0), (-2, 0)]

In [5]:
def is_valid(board, x, y, dx, dy):
    if 0 <= x + dx < 7 and 0 <= y + dy < 7 and board[x][y] == 1:
        if board[x + dx // 2][y + dy // 2] == 1 and board[x + dx][y + dy] == 0:
            return True
    return False

def apply_move(board, x, y, dx, dy):
    new_board = [row[:] for row in board]
    new_board[x][y] = 0
    new_board[x + dx // 2][y + dy // 2] = 0
    new_board[x + dx][y + dy] = 1
    return new_board

In [6]:
def num_marbles(board):
    return sum(row.count(1) for row in board)

In [7]:
def manhattan_heuristic(board):
    center = (3, 3)
    distance = 0
    for i in range(len(board)):
        for j in range(len(board[i])):
            if board[i][j] == 1:
                distance += abs(i - center[0]) + abs(j - center[1])
    return distance

def exponential_heuristic(board):
    center = (3, 3)
    exp_distance = 0
    for i in range(len(board)):
        for j in range(len(board[i])):
            if board[i][j] == 1:
                d = max(abs(i - center[0]), abs(j - center[1]))
                exp_distance += 2 ** d
    return exp_distance

In [8]:
def get_successors(node):
    successors = []
    for x in range(7):
        for y in range(7):
            if node.board[x][y] == 1:
                for dx, dy in directions:
                    if is_valid(node.board, x, y, dx, dy):
                        new_board = apply_move(node.board, x, y, dx, dy)
                        successor_node = Node(new_board, g=node.g+1, path=node.path + [(x, y, x + dx, y + dy)])
                        successors.append(successor_node)
    return successors

def is_goal_state(board):
    return num_marbles(board) == 1 and board[3][3] == 1

In [9]:
def best_first_search(start_board, heuristic_func):
    start_node = Node(start_board)
    frontier = [(heuristic_func(start_board), start_node)]
    visited = set()
    nodes_expanded = 0

    while frontier:
        _, current_node = heapq.heappop(frontier)
        nodes_expanded += 1

        if is_goal_state(current_node.board):
            return current_node.path, nodes_expanded

        board_tuple = tuple(map(tuple, current_node.board))
        if board_tuple not in visited:
            visited.add(board_tuple)
            for successor in get_successors(current_node):
                heapq.heappush(frontier, (heuristic_func(successor.board), successor))

    return None, nodes_expanded

In [10]:
def a_star_search(start_board, heuristic_func):
    start_node = Node(start_board, h=heuristic_func(start_board))
    frontier = [(start_node.f, start_node)]
    visited = set()
    nodes_expanded = 0

    while frontier:
        _, current_node = heapq.heappop(frontier)
        nodes_expanded += 1

        if is_goal_state(current_node.board):
            return current_node.path, nodes_expanded

        board_tuple = tuple(map(tuple, current_node.board))
        if board_tuple not in visited:
            visited.add(board_tuple)
            for successor in get_successors(current_node):
                successor.h = heuristic_func(successor.board)
                successor.f = successor.g + successor.h
                heapq.heappush(frontier, (successor.f, successor))

    return None, nodes_expanded

In [11]:
def uniform_cost_search(start_board):
    start_node = Node(start_board)
    frontier = [(0, start_node)]
    visited = set()
    nodes_expanded = 0

    while frontier:
        _, current_node = heapq.heappop(frontier)
        nodes_expanded += 1

        if is_goal_state(current_node.board):
            return current_node.path, nodes_expanded

        board_tuple = tuple(map(tuple, current_node.board))
        if board_tuple not in visited:
            visited.add(board_tuple)
            for successor in get_successors(current_node):
                heapq.heappush(frontier, (successor.g, successor))

    return None, nodes_expanded

In [12]:
def compare_algorithms(initial_board):
    algorithms = [
        ("Best First Search (Manhattan)", lambda b: best_first_search(b, manhattan_heuristic)),
        ("Best First Search (Exponential)", lambda b: best_first_search(b, exponential_heuristic)),
        ("A* (Manhattan)", lambda b: a_star_search(b, manhattan_heuristic)),
        ("A* (Exponential)", lambda b: a_star_search(b, exponential_heuristic)),
        ("Priority Queue Search", lambda b: uniform_cost_search(b))
    ]

    results = []

    for name, algo in algorithms:
        start_time = time.time()
        solution, nodes_expanded = algo(initial_board)
        end_time = time.time()

        results.append({
            "name": name,
            "solution": solution,
            "nodes_expanded": nodes_expanded,
            "time": end_time - start_time
        })

    return results

In [ ]:
initial_board = [
    [-1, -1, 1, 1, 1, -1, -1],
    [-1, -1, 1, 1, 1, -1, -1],
    [1, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 0, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 1],
    [-1, -1, 1, 1, 1, -1, -1],
    [-1, -1, 1, 1, 1, -1, -1]
]

results = compare_algorithms(initial_board)

for result in results:
    print(f"\n{result['name']}:")
    print(f"  Solution found: {'Yes' if result['solution'] else 'No'}")
    print(f"  Nodes expanded: {result['nodes_expanded']}")
    print(f"  Time taken: {result['time']:.2f} seconds")
    if result['solution']:
        print(f"  Solution length: {len(result['solution'])} moves")
        print(f"  Solution path: {result['solution']}")